# 🧩 Chapter 9: 3D Shape Detection & Segmentation

Detecting geometric shapes in point clouds is essential for reverse engineering and scene understanding. In this chapter, we explore two fundamental algorithms:
1.  **RANSAC (Random Sample Consensus)**: Robustly fitting mathematical models (planes, spheres) to noisy data.
2.  **Region Growing**: Segmenting objects based on surface connectivity and smoothness.

**Objectives:**
*   Implement RANSAC from scratch to find the dominant plane (ground/floor).
*   Use Open3D's RANSAC for efficiency.
*   Implement Region Growing to separate distinct objects.

In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
import time

## 1. Load Data

We use a dataset containing a desk setup, which has clear planar surfaces.

In [ ]:
file_path = "../DATA/the_researcher_desk.ply"

try:
    pcd = o3d.io.read_point_cloud(file_path)
    print(f"Loaded {pcd}")
    o3d.visualization.draw_geometries([pcd], window_name="Original Cloud")
except Exception as e:
    print(f"Error loading file: {e}")

## 2. RANSAC Plane Fitting

RANSAC iteratively selects random subsets of points to fit a model and counts how many other points support that model (inliers).

**Key Parameters:**
*   `distance_threshold`: Max distance a point can be from the plane to be considered an inlier.
*   `ransac_n`: Number of points to sample (3 for a plane).
*   `num_iterations`: How many times to try.

In [ ]:
# Open3D Implementation
distance_threshold = 0.02 # 2 cm
ransac_n = 3
num_iterations = 1000

start = time.time()
plane_model, inliers = pcd.segment_plane(distance_threshold=distance_threshold,
                                         ransac_n=ransac_n,
                                         num_iterations=num_iterations)
[a, b, c, d] = plane_model
print(f"Plane Equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
print(f"RANSAC took {time.time() - start:.4f} seconds.")

# Extract Inliers (The Plane)
inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0]) # Red Plane

# Extract Outliers (Everything else)
outlier_cloud = pcd.select_by_index(inliers, invert=True)

# Visualize
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud], window_name="RANSAC Result")

## 3. Region Growing Segmentation

RANSAC finds *one* global shape. To find *multiple* distinct objects (e.g., monitor, chair, keyboard) based on local connectivity and smoothness, we use Region Growing.

**Logic:**
1.  Pick a seed point.
2.  Check its neighbors.
3.  If a neighbor is close enough (distance) and has a similar normal vector (angle), add it to the region.
4.  Repeat until no more neighbors fit.

In [ ]:
# Ensure normals are computed
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Region growing isn't natively exposed in Open3D Python API as a single function call typically,
# usually we use the ClusterDBSCAN for density based or implement a custom one.
# Here we use Open3D's DBSCAN as a robust alternative for object segmentation.

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd.cluster_dbscan(eps=0.05, min_points=10, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

# Colorize clusters
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0 # Noise is black
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

o3d.visualization.draw_geometries([pcd], window_name="DBSCAN Segmentation")

## 4. Custom Region Growing (Educational)

For strict control over *smoothness* (normal angle), we can implement a basic region grower. This is often better for planar/smooth surfaces than DBSCAN (which only looks at density).

In [ ]:
def simple_region_growing(pcd, distance_threshold=0.05, angle_threshold_deg=10):
    points = np.asarray(pcd.points)
    normals = np.asarray(pcd.normals)
    N = len(points)
    
    # Tree for fast lookup
    tree = o3d.geometry.KDTreeFlann(pcd)
    
    visited = np.zeros(N, dtype=bool)
    labels = -1 * np.ones(N, dtype=int)
    cluster_id = 0
    
    angle_threshold_rad = np.deg2rad(angle_threshold_deg)
    
    for i in range(N):
        if visited[i]:
            continue
            
        # Start new cluster
        seed_queue = [i]
        visited[i] = True
        labels[i] = cluster_id
        
        while seed_queue:
            curr_idx = seed_queue.pop(0)
            curr_normal = normals[curr_idx]
            
            # Find neighbors
            [k, idx, _] = tree.search_radius_vector_3d(points[curr_idx], distance_threshold)
            
            for neighbor_idx in idx:
                if not visited[neighbor_idx]:
                    # Check smoothness (angle between normals)
                    # Dot product close to 1 means parallel
                    dot = np.dot(curr_normal, normals[neighbor_idx])
                    # Clip to avoid numerical errors
                    dot = np.clip(dot, -1.0, 1.0)
                    angle = np.arccos(dot)
                    
                    if angle < angle_threshold_rad:
                        visited[neighbor_idx] = True
                        labels[neighbor_idx] = cluster_id
                        seed_queue.append(neighbor_idx)
                        
        cluster_id += 1
        
    return labels

# Run custom region growing (on a downsampled cloud to be fast)
pcd_down = pcd.voxel_down_sample(voxel_size=0.02)
pcd_down.estimate_normals()
labels_rg = simple_region_growing(pcd_down, distance_threshold=0.03, angle_threshold_deg=15)
print(f"Found {labels_rg.max() + 1} smooth regions.")

# Visualize
max_label = labels_rg.max()
colors = plt.get_cmap("tab20")(labels_rg / (max_label if max_label > 0 else 1))
colors[labels_rg < 0] = 0
pcd_down.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd_down], window_name="Smootheness-based Region Growing")